<a href="https://colab.research.google.com/github/dkurbatovv/Python/blob/main/PyCaret_(bad_results_(_)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycaret

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
!mkdir -p ~/.kaggle/
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download -c house-prices-advanced-regression-techniques

In [ ]:
! unzip house-prices-advanced-regression-techniques.zip

In [ ]:
from pycaret.regression import *

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv('/content/train.csv')
test  = pd.read_csv('/content/test.csv')
sample= pd.read_csv('/content/sample_submission.csv')

In [ ]:
sns.set_style("white")
sns.set_color_codes(palette='deep')
f, ax = plt.subplots(figsize=(8, 7))
#Check the new distribution 
sns.distplot(train['SalePrice'], color="b");
ax.xaxis.grid(False)
ax.set(ylabel="Frequency")
ax.set(xlabel="SalePrice")
ax.set(title="SalePrice distribution")
sns.despine(trim=True, left=True)
plt.show()

In [ ]:
# Finding numeric features
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric = []
for i in train.columns:
    if train[i].dtype in numeric_dtypes:
        if i in ['TotalSF', 'Total_Bathrooms','Total_porch_sf','haspool','hasgarage','hasbsmt','hasfireplace']:
            pass
        else:
            numeric.append(i)     
# visualising some more outliers in the data values
fig, axs = plt.subplots(ncols=2, nrows=0, figsize=(12, 120))
plt.subplots_adjust(right=2)
plt.subplots_adjust(top=2)
sns.color_palette("husl", 8)
for i, feature in enumerate(list(train[numeric]), 1):
    if(feature=='MiscVal'):
        break
    plt.subplot(len(list(numeric)), 3, i)
    sns.scatterplot(x=feature, y='SalePrice', hue='SalePrice', palette='Blues', data=train)
        
    plt.xlabel('{}'.format(feature), size=15,labelpad=12.5)
    plt.ylabel('SalePrice', size=15, labelpad=12.5)
    
    for j in range(2):
        plt.tick_params(axis='x', labelsize=12)
        plt.tick_params(axis='y', labelsize=12)
    
    plt.legend(loc='best', prop={'size': 10})
        
plt.show()

In [ ]:
train_ID = train['Id']
test_ID = test['Id']
train.drop(['Id'], axis=1, inplace=True)
test.drop(['Id'], axis=1, inplace=True)
train.shape, test.shape

In [ ]:
train["SalePrice"] = np.log1p(train["SalePrice"])



In [ ]:
sns.set_style("white")
sns.set_color_codes(palette='deep')
f, ax = plt.subplots(figsize=(8, 7))
#Check the new distribution 
sns.distplot(train['SalePrice'] , fit=norm, color="b");

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(train['SalePrice'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
ax.xaxis.grid(False)
ax.set(ylabel="Frequency")
ax.set(xlabel="SalePrice")
ax.set(title="SalePrice distribution")
sns.despine(trim=True, left=True)

plt.show()

In [ ]:
train.drop(train[(train['OverallQual']<5) & (train['SalePrice']>200000)].index, inplace=True)
train.drop(train[(train['GrLivArea']>4500) & (train['SalePrice']<300000)].index, inplace=True)
train.reset_index(drop=True, inplace=True)

In [ ]:
train['MSSubClass'] = train['MSSubClass'].apply(str)
train['YrSold'] = train['YrSold'].astype(str)
train['MoSold'] = train['MoSold'].astype(str)

In [ ]:
test['MSSubClass'] = test['MSSubClass'].apply(str)
test['YrSold'] = test['YrSold'].astype(str)
test['MoSold'] = test['MoSold'].astype(str)

In [ ]:
def handle_missing(features):
    # the data description states that NA refers to typical ('Typ') values
    features['Functional'] = features['Functional'].fillna('Typ')
    # Replace the missing values in each of the columns below with their mode
    features['Electrical'] = features['Electrical'].fillna("SBrkr")
    features['KitchenQual'] = features['KitchenQual'].fillna("TA")
    features['Exterior1st'] = features['Exterior1st'].fillna(features['Exterior1st'].mode()[0])
    features['Exterior2nd'] = features['Exterior2nd'].fillna(features['Exterior2nd'].mode()[0])
    features['SaleType'] = features['SaleType'].fillna(features['SaleType'].mode()[0])
    features['MSZoning'] = features.groupby('MSSubClass')['MSZoning'].transform(lambda x: x.fillna(x.mode()[0]))
    
    # the data description stats that NA refers to "No Pool"
    features["PoolQC"] = features["PoolQC"].fillna("None")
    # Replacing the missing values with 0, since no garage = no cars in garage
    for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
        features[col] = features[col].fillna(0)
    # Replacing the missing values with None
    for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
        features[col] = features[col].fillna('None')
    # NaN values for these categorical basement features, means there's no basement
    for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
        features[col] = features[col].fillna('None')
        
    # Group the by neighborhoods, and fill in missing value by the median LotFrontage of the neighborhood
    features['LotFrontage'] = features.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

    # We have no particular intuition around how to fill in the rest of the categorical features
    # So we replace their missing values with None
    objects = []
    for i in features.columns:
        if features[i].dtype == object:
            objects.append(i)
    features.update(features[objects].fillna('None'))
        
    # And we do the same thing for numerical features, but this time with 0s
    numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    numeric = []
    for i in features.columns:
        if features[i].dtype in numeric_dtypes:
            numeric.append(i)
    features.update(features[numeric].fillna(0))    
    return features

train = handle_missing(train)

test = handle_missing(test)

In [ ]:
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric = []
for i in train.columns:
    if train[i].dtype in numeric_dtypes:
        numeric.append(i)

In [ ]:
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric = []
for i in test.columns:
    if test[i].dtype in numeric_dtypes:
        numeric.append(i)

In [ ]:
from scipy.stats import skew, norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

skew_features = train[numeric].apply(lambda x: skew(x)).sort_values(ascending=False)

high_skew = skew_features[skew_features > 0.5]
skew_index = high_skew.index

print("There are {} numerical features with Skew > 0.5 :".format(high_skew.shape[0]))
skewness = pd.DataFrame({'Skew' :high_skew})
skew_features.head(10)

In [ ]:
skew_features = test[numeric].apply(lambda x: skew(x)).sort_values(ascending=False)

high_skew = skew_features[skew_features > 0.5]
skew_index = high_skew.index

print("There are {} numerical features with Skew > 0.5 :".format(high_skew.shape[0]))
skewness = pd.DataFrame({'Skew' :high_skew})
skew_features.head(10)

In [ ]:
for i in skew_index:
    train[i] = boxcox1p(train[i], boxcox_normmax(train[i] + 1))

In [ ]:
for i in skew_index:
    test[i] = boxcox1p(test[i], boxcox_normmax(test[i] + 1))

In [ ]:
train['BsmtFinType1_Unf'] = 1*(train['BsmtFinType1'] == 'Unf')
train['HasWoodDeck'] = (train['WoodDeckSF'] == 0) * 1
train['HasOpenPorch'] = (train['OpenPorchSF'] == 0) * 1
train['HasEnclosedPorch'] = (train['EnclosedPorch'] == 0) * 1
train['Has3SsnPorch'] = (train['3SsnPorch'] == 0) * 1
train['HasScreenPorch'] = (train['ScreenPorch'] == 0) * 1
train['YearsSinceRemodel'] = train['YrSold'].astype(int) - train['YearRemodAdd'].astype(int)
train['Total_Home_Quality'] = train['OverallQual'] + train['OverallCond']
train = train.drop(['Utilities', 'Street', 'PoolQC',], axis=1)
train['TotalSF'] = train['TotalBsmtSF'] + train['1stFlrSF'] + train['2ndFlrSF']
train['YrBltAndRemod'] = train['YearBuilt'] + train['YearRemodAdd']

train['Total_sqr_footage'] = (train['BsmtFinSF1'] + train['BsmtFinSF2'] +
                                 train['1stFlrSF'] + train['2ndFlrSF'])
train['Total_Bathrooms'] = (train['FullBath'] + (0.5 * train['HalfBath']) +
                               train['BsmtFullBath'] + (0.5 * train['BsmtHalfBath']))
train['Total_porch_sf'] = (train['OpenPorchSF'] + train['3SsnPorch'] +
                              train['EnclosedPorch'] + train['ScreenPorch'] +
                              train['WoodDeckSF'])
train['TotalBsmtSF'] = train['TotalBsmtSF'].apply(lambda x: np.exp(6) if x <= 0.0 else x)
train['2ndFlrSF'] = train['2ndFlrSF'].apply(lambda x: np.exp(6.5) if x <= 0.0 else x)
train['GarageArea'] = train['GarageArea'].apply(lambda x: np.exp(6) if x <= 0.0 else x)
train['GarageCars'] = train['GarageCars'].apply(lambda x: 0 if x <= 0.0 else x)
train['LotFrontage'] = train['LotFrontage'].apply(lambda x: np.exp(4.2) if x <= 0.0 else x)
train['MasVnrArea'] = train['MasVnrArea'].apply(lambda x: np.exp(4) if x <= 0.0 else x)
train['BsmtFinSF1'] = train['BsmtFinSF1'].apply(lambda x: np.exp(6.5) if x <= 0.0 else x)

train['haspool'] = train['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
train['has2ndfloor'] = train['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
train['hasgarage'] = train['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
train['hasbsmt'] = train['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
train['hasfireplace'] = train['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
test['BsmtFinType1_Unf'] = 1*(test['BsmtFinType1'] == 'Unf')
test['HasWoodDeck'] = (test['WoodDeckSF'] == 0) * 1
test['HasOpenPorch'] = (test['OpenPorchSF'] == 0) * 1
test['HasEnclosedPorch'] = (test['EnclosedPorch'] == 0) * 1
test['Has3SsnPorch'] = (test['3SsnPorch'] == 0) * 1
test['HasScreenPorch'] = (test['ScreenPorch'] == 0) * 1
test['YearsSinceRemodel'] = test['YrSold'].astype(int) - test['YearRemodAdd'].astype(int)
test['Total_Home_Quality'] = test['OverallQual'] + test['OverallCond']
test = test.drop(['Utilities', 'Street', 'PoolQC',], axis=1)
test['TotalSF'] = test['TotalBsmtSF'] + test['1stFlrSF'] + test['2ndFlrSF']
test['YrBltAndRemod'] = test['YearBuilt'] + test['YearRemodAdd']

test['Total_sqr_footage'] = (test['BsmtFinSF1'] + test['BsmtFinSF2'] +
                                 test['1stFlrSF'] + test['2ndFlrSF'])
test['Total_Bathrooms'] = (test['FullBath'] + (0.5 * test['HalfBath']) +
                               test['BsmtFullBath'] + (0.5 * test['BsmtHalfBath']))
test['Total_porch_sf'] = (test['OpenPorchSF'] + test['3SsnPorch'] +
                              test['EnclosedPorch'] + test['ScreenPorch'] +
                              test['WoodDeckSF'])
test['TotalBsmtSF'] = test['TotalBsmtSF'].apply(lambda x: np.exp(6) if x <= 0.0 else x)
test['2ndFlrSF'] = test['2ndFlrSF'].apply(lambda x: np.exp(6.5) if x <= 0.0 else x)
test['GarageArea'] = test['GarageArea'].apply(lambda x: np.exp(6) if x <= 0.0 else x)
test['GarageCars'] = test['GarageCars'].apply(lambda x: 0 if x <= 0.0 else x)
test['LotFrontage'] = test['LotFrontage'].apply(lambda x: np.exp(4.2) if x <= 0.0 else x)
test['MasVnrArea'] = test['MasVnrArea'].apply(lambda x: np.exp(4) if x <= 0.0 else x)
test['BsmtFinSF1'] = test['BsmtFinSF1'].apply(lambda x: np.exp(6.5) if x <= 0.0 else x)

test['haspool'] = test['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
test['has2ndfloor'] = test['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
test['hasgarage'] = test['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
test['hasbsmt'] = test['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
test['hasfireplace'] = test['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
def logs(res, ls):
    m = res.shape[1]
    for l in ls:
        res = res.assign(newcol=pd.Series(np.log(1.01+res[l])).values)   
        res.columns.values[m] = l + '_log'
        m += 1
    return res

log_features = ['LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF',
                 'TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea',
                 'BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr',
                 'TotRmsAbvGrd','Fireplaces','GarageCars','GarageArea','WoodDeckSF','OpenPorchSF',
                 'EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal','YearRemodAdd','TotalSF']

train = logs(train, log_features)

test = logs(test, log_features)

In [ ]:
def squares(res, ls):
    m = res.shape[1]
    for l in ls:
        res = res.assign(newcol=pd.Series(res[l]*res[l]).values)   
        res.columns.values[m] = l + '_sq'
        m += 1
    return res 

squared_features = ['YearRemodAdd', 'LotFrontage_log', 
              'TotalBsmtSF_log', '1stFlrSF_log', '2ndFlrSF_log', 'GrLivArea_log',
              'GarageCars_log', 'GarageArea_log']

train = squares(train, squared_features)

test = squares(test, squared_features)

In [ ]:
train = pd.get_dummies(train).reset_index(drop=True)
train.shape

In [ ]:
test = pd.get_dummies(test).reset_index(drop=True)

In [ ]:
train = train.loc[:,~train.columns.duplicated()]

test = test.loc[:,~test.columns.duplicated()]

In [ ]:
reg = setup(data = train, 
             target = 'SalePrice',
             numeric_imputation = 'mean',
             normalize = True,
             ignore_features = ['Condition2_RRAe', 'Condition2_RRAn', 'Condition2_RRNn', 'HouseStyle_2.5Fin', 
                               'RoofMatl_Membran', 'RoofMatl_Metal', 'RoofMatl_Roll', 'Exterior1st_ImStucc', 
                               'Exterior1st_Stone', 'Exterior2nd_Other', 'Heating_Floor', 'Heating_OthW', 
                               'Electrical_Mix', 'GarageQual_Ex', 'MiscFeature_TenC'],
             silent = True)

In [ ]:
compare_models()

In [ ]:
br = create_model('br')
lightgbm = create_model('lightgbm')
rf = create_model('rf')

In [ ]:
tuned_lgb = tune_model(br)
tuned_lgb = tune_model(lightgbm)
tuned_lgb = tune_model(rf)

In [ ]:
print(train.shape, test.shape)

In [ ]:
predictions = predict_model(tuned_lgb, data = test)
sample['SalePrice'] = predictions['Label']
sample.to_csv('submission_house_price.csv', index=False)
sample.head()